In [1]:
%load_ext autoreload
%autoreload 2

### Set AWS Credentials
!aws configure list-profiles
!export AWS_PROFILE=dexter
%env AWS_PROFILE=dexter
!aws sts get-caller-identity

Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: argument subcommand: Invalid choice, valid choices are:

list                                     | get                                     
set                                      | add-model                               
env: AWS_PROFILE=dexter
{
    "UserId": "AIDA3IY7YAJ5E36J4DXW3",
    "Account": "774771311226",
    "Arn": "arn:aws:iam::774771311226:user/dvdmlee"
}


In [2]:
import boto3

from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings

bedrock = boto3.client(
    service_name='bedrock',
    region_name='us-east-1',
    endpoint_url='https://bedrock.us-east-1.amazonaws.com'
)

embeddings = BedrockEmbeddings(client=bedrock)

llm = Bedrock(
    client=bedrock, 
    model_id="amazon.titan-tg1-large"
    )

In [4]:
from code.prompts import QuestionAnswerTemplate

In [5]:
qat = QuestionAnswerTemplate(llm)

In [10]:
qat.prompt, qat.template_string

(PromptTemplate(input_variables=['query', 'sources'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Organize your answer into logical chunks as needed and be as clear and concise as possible. Return an answer is 1-3 sentences long. \n\n        Context: ```\n        {sources}\n        ```\n\n        Question: ```\n        {query}\n        ```\n\n        Helpful Answer:\n        ", template_format='f-string', validate_template=True),
 "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Organize your answer into logical chunks as needed and be as clear and concise as possible. Return an answer is 1-3 sentences long. \n\n        Context: ```\n        {sources}\n        ```\n\n        Question: ```\n        {query}

### parser test

takeaway is parser does not work well with Amazon...

https://python.langchain.com/docs/modules/model_io/output_parsers/

In [39]:
# from langchain.output_parsers import PydanticOutputParser

# from pydantic import BaseModel, Field, validator

# class ContactSummary(BaseModel):
#     summary: str = Field(description="Summary of the customer ticket")
#     key_details: str = Field(description="Key details of the customer ticket")

# class ContactSummary(BaseModel):
#     # summary: str = Field(description="Summary of the customer ticket")
#     # key_details: str = Field(description="Key details of the customer ticket")
#     name: str = Field(description="Name of the customer")
#     email: str = Field(description="Email of the customer")
#     address: str = Field(description="Address of the customer")
#     phone_number: str = Field(description="Phone number of the customer")

# # Set up a parser + inject instructions into the prompt template.
# parser = PydanticOutputParser(pydantic_object=ContactSummary)

In [40]:
# from langchain import PromptTemplate

# pt = PromptTemplate(
#     template='Summarize the customer ticket and provide key identification details such as name, email, ids, etc.\n{format_instructions}\nTicket:```{ticket}```\n',
#     input_variables=['ticket'],
#     partial_variables={'format_instructions': parser.get_format_instructions()}
# )

# pt = PromptTemplate(
#     template='''Extract the name, email, address and phone number from the provided ticket.
    
#     Format instructions: {format_instructions}
    
#     Ticket:```{ticket}```\n''',
#     input_variables=['ticket'],
#     partial_variables={'format_instructions': parser.get_format_instructions()}
# )

# ticket = '''Dear Shnoogy Support Team,

# Help! I'm in a tax nightmare! The 1099 tax form I received as a seller on Shnoogy is totally messed up. My name is Emma Thompson (Seller ID: #SHN123) and I am beyond frustrated. The reported sales amount is way off, and it's going to create huge problems with my taxes. I need this fixed urgently!

# I don't have time for back-and-forth. Please investigate the discrepancies ASAP and send me an accurate revised 1099 form immediately. My financial obligations are at stake, and I rely on Shnoogy to get this right.

# Contact me at emma.thompson@example.com or (555) 789-1234. Address: 456 Elm Street, Anytown, USA.

# I expect swift action and resolution. This cannot wait. Don't let me down!

# Sincerely,

# Emma Thompson'''

# _input = pt.format_prompt(ticket=ticket)
# _input

In [41]:
# _input.to_string()

In [42]:
# output = llm(_input.to_string())
# print(output)

In [43]:
# parser.parse(output)

### Memory

In [44]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [45]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [46]:
conversation.predict(input='Hi, my name is David')



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is David
AI:

> Finished chain.


' Hello! My name is Atlas. I appreciate you giving me your name, David.'

In [47]:
conversation.predict(input='My favorite sports team is Arsenal')



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is David
AI:  Hello! My name is Atlas. I appreciate you giving me your name, David.
Human: My favorite sports team is Arsenal
AI:

> Finished chain.


' Arsenal is an English professional football club located in Islington, London, England. What is your favorite team about them?'

In [48]:
conversation.predict(input='Whats a good city for me to visit?')



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is David
AI:  Hello! My name is Atlas. I appreciate you giving me your name, David.
Human: My favorite sports team is Arsenal
AI:  Arsenal is an English professional football club located in Islington, London, England. What is your favorite team about them?
Human: Whats a good city for me to visit?
AI:

> Finished chain.


" I don't know. I'm not sure of your tastes or what you're looking for."

In [49]:
conversation.predict(input='Whats a good city for me to visit to watch a soccer game?')



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is David
AI:  Hello! My name is Atlas. I appreciate you giving me your name, David.
Human: My favorite sports team is Arsenal
AI:  Arsenal is an English professional football club located in Islington, London, England. What is your favorite team about them?
Human: Whats a good city for me to visit?
AI:  I don't know. I'm not sure of your tastes or what you're looking for.
Human: Whats a good city for me to visit to watch a soccer game?
AI:

> Finished chain.


' A good city to visit to watch a soccer game would be Manchester, England. Manchester is home to two major soccer clubs: Manchester City F.C. and Manchester United F.C.'

In [53]:
print(memory.load_memory_variables({}))

{'history': "Human: Hi, my name is David\nAI:  Hello! My name is Atlas. I appreciate you giving me your name, David.\nHuman: My favorite sports team is Arsenal\nAI:  Arsenal is an English professional football club located in Islington, London, England. What is your favorite team about them?\nHuman: Whats a good city for me to visit?\nAI:  I don't know. I'm not sure of your tastes or what you're looking for.\nHuman: Whats a good city for me to visit to watch a soccer game?\nAI:  A good city to visit to watch a soccer game would be Manchester, England. Manchester is home to two major soccer clubs: Manchester City F.C. and Manchester United F.C."}


In [52]:
print(memory.buffer)

Human: Hi, my name is David
AI:  Hello! My name is Atlas. I appreciate you giving me your name, David.
Human: My favorite sports team is Arsenal
AI:  Arsenal is an English professional football club located in Islington, London, England. What is your favorite team about them?
Human: Whats a good city for me to visit?
AI:  I don't know. I'm not sure of your tastes or what you're looking for.
Human: Whats a good city for me to visit to watch a soccer game?
AI:  A good city to visit to watch a soccer game would be Manchester, England. Manchester is home to two major soccer clubs: Manchester City F.C. and Manchester United F.C.


In [55]:
memory = ConversationBufferMemory()
memory.save_context({'input': 'Hi'}, {'output': 'Whats up dog'})
memory.save_context({'input': 'nm jc jc'}, {'output': 'oh fs thats a vibe'})

In [57]:
print(memory.load_memory_variables({}))

{'history': 'Human: Hi\nAI: Whats up dog\nHuman: nm jc jc\nAI: oh fs thats a vibe'}


In [59]:
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

conversation.predict(input='yup yup true dat')



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi
AI: Whats up dog
Human: nm jc jc
AI: oh fs thats a vibe
Human: yup yup true dat
AI:

> Finished chain.


' I don\'t know what "nm jc jc" means.'

In [60]:
from langchain.memory import ConversationBufferWindowMemory

In [65]:
memory = ConversationBufferWindowMemory(k=2)

memory.save_context({'input': 'Hi my name is David'}, {'output': 'Whats up homie Dave'})
memory.save_context({'input': 'not much just chilling just chilling'}, {'output': 'oh for sure. thats a vibe'})

print(memory.load_memory_variables({}))

{'history': 'Human: Hi my name is David\nAI: Whats up homie Dave\nHuman: not much just chilling just chilling\nAI: oh for sure. thats a vibe'}


In [66]:
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

conversation.predict(input='whats my name homie?')



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi my name is David
AI: Whats up homie Dave
Human: not much just chilling just chilling
AI: oh for sure. thats a vibe
Human: whats my name homie?
AI:

> Finished chain.


' Your name is David. There are no other details associated with your account that would help me answer this question further.'

In [70]:
from langchain.memory import ConversationTokenBufferMemory

In [71]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

/Users/dvdmlee/Projects/_bedrock/venv_bedrock/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [72]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

In [73]:
from langchain.memory import ConversationSummaryBufferMemory

In [95]:
# create a long string
restate = '''I understand. it seems like the name and address need to be updated. and that the financials must be double checked.is that correct?'''

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hi I need help"}, {"output": "Okay what can i help you with?"})
memory.save_context({"input": "I just receieved my 1099. It does not look correct. Can you fix it?"},
                    {"output": "Sure what looks wrong with it?"})
memory.save_context({"input": "The details are not right. First off, my name is David Boodycheeks not David Butticheeg. Second, the address is wrong. I moved from Florida to Texas. My new address is 123 Main Street, Austin TX. Lastly, the amount does not look correct. It says I only made 300k when in reality I made close to 500k."}, 
                    {"output": f"{restate}"})
memory.save_context({'input': 'that is correct.'}, {'output': 'okay we will update the information and send it out.'})

In [96]:
memory.load_memory_variables({})

{'history': 'System: \nThe human asks for help with their 1099. The AI agrees to help and asks for more information. The human explains that the name, address, and amount are all wrong. The AI agrees to make the changes.\nAI: I understand. it seems like the name and address need to be updated. and that the financials must be double checked.is that correct?\nHuman: that is correct.\nAI: okay we will update the information and send it out.'}

In [97]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [98]:
conversation.predict(input="great so just to make sure you understand, what are you going to correct?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human asks for help with their 1099. The AI agrees to help and asks for more information. The human explains that the name, address, and amount are all wrong. The AI agrees to make the changes.
AI: I understand. it seems like the name and address need to be updated. and that the financials must be double checked.is that correct?
Human: that is correct.
AI: okay we will update the information and send it out.
Human: great so just to make sure you understand, what are you going to correct?
AI:

> Finished chain.


' I understand that the current 1099 issued needs to be corrected because the name, address, and amount are incorrect. I will update the name, address, and amount then resubmit the 1099.'

### entity memory

In [104]:
from langchain.memory import ConversationEntityMemory

memory = ConversationEntityMemory(llm=llm, return_messages=True)
_input = {"input": "Bukayo Saka is an Arsenal player."}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " Interesting. How long has been on Arsenal?"}
)

In [105]:
memory.load_memory_variables({"input": 'who is Bukayo Saka'})


{'history': [HumanMessage(content='Bukayo Saka is an Arsenal player.', additional_kwargs={}, example=False),
  AIMessage(content=' Interesting. How long has been on Arsenal?', additional_kwargs={}, example=False)],
 'entities': {'Bukayo Saka': 'Bukayo Saka is a player for the Arsenal.'}}

In [106]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any

In [117]:
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [118]:
conversation.predict(input="Bukayo Saka is a soccer player on Arsenal. He wears the number 7 jersey.")



> Entering new  chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to e

" Yes. He was born on March 25th, 2001 in London, England. He plays as a winger for both Arsenal and the England national team. Saka is a fast and skillful player who is known for his excellent dribbling and crossing abilities. He began his career with Arsenal's youth academy and made his senior debut for the club in 2018. Since then, he has become a regular starter for Arsenal and has also represented England at the international level. In 2021, he helped Arsenal win the FA Cup and was named the club's young player of the year."

In [119]:
conversation.memory.entity_store.store

{'Bukayo Saka': 'He was born on March 25th, 2001 in London, England. He plays as a winger for both Arsenal and the England national team. Saka is a fast and skillful player who is known for his excellent dribbling and crossing abilities.',
 'Arsenal': "Bukayo Saka is a soccer player on Arsenal who wears the number 7 jersey and was born on March 25th, 2001 in London, England. He plays as a winger for both Arsenal and the England national team and is known for his excellent dribbling and crossing abilities. Saka helped Arsenal win the FA Cup in 2021 and was named the club's young player of the year."}

In [120]:
conversation.predict(input="Arsenal finished in second place in the Premier League in 2023 after Manchester City.")




> Entering new  chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to e

' This is true. Manchester City won the Premier League title in 2023 with a total of 93 points, while Arsenal finished in second place with 86 points. It was a close race between the two teams, with Manchester City winning the title by just three points.'

In [121]:
conversation.memory.entity_store.store

{'Bukayo Saka': 'He was born on March 25th, 2001 in London, England. He plays as a winger for both Arsenal and the England national team. Saka is a fast and skillful player who is known for his excellent dribbling and crossing abilities.',
 'Arsenal': 'Bukayo Saka is a soccer player on Arsenal who wears the number 7 jersey and was born on March 25th, 2001 in London, England. He plays as a winger for both Arsenal and the England national team and is known for his excellent dribbling and crossing abilities. Arsenal finished in second place in the Premier League in 2023, with Manchester City winning the title with 93 points, while Arsenal finished with 86 points. It was a close race between the two teams.',
 'Manchester City': 'Manchester City won the Premier League title in 2023 with a total of 93 points, while Arsenal finished in second place with 86 points.'}

In [122]:
conversation.predict(input="Arsenal just signed Declan Rice from West Ham. Declan Rice is good friends with Bukayo Saka from the England national team. Saka scored a hat trick in his last game for England.")






> Entering new  chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to e

" Yes. I am aware of that. It is exciting to see how these new signings will impact the team's performance this season. Arsenal has made some promising signings this off-season, and fans are hopeful that the team will be able to challenge for the Premier League title once again. With the addition of players like Saka and Rice, along with the continued development of the club's existing talent, Arsenal has the potential to be a strong competitor in the league. It will be interesting to see how the team performs in the upcoming season."

In [123]:
conversation.memory.entity_store.store

{'Bukayo Saka': 'He was born on March 25th, 2001 in London, England. He plays as a winger for both Arsenal and the England national team. Saka is a fast and skillful player who is known for his excellent dribbling and crossing abilities, and he recently scored a hat trick in his last game for England. Recently, Arsenal signed Declan Rice from West Ham, and the two players are good friends. Fans are hopeful that these new signings will help Arsenal challenge for the Premier League title this season.',
 'Arsenal': "Arsenal has signed Declan Rice from West Ham, who is good friends with Bukayo Saka from the England national team. Saka scored a hat trick in his last game for England. This off-season, Arsenal has made promising signings that fans are hopeful will help the team challenge for the Premier League title once again. With the addition of players like Saka and Rice, along with the continued development of the club's existing talent, Arsenal has the potential to be a strong competito

In [124]:
conversation.predict(input="What do you know about Bukayo Saka?")





> Entering new  chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to e

" Bukayo Saka was born on March 25th, 2001, in London, England. He plays as a winger for both Arsenal and the England national team. He's known for his excellent dribbling and crossing abilities. Recently, he scored a hat trick in his last game for England. Arsenal recently signed Declan Rice from West Ham, and the two players are good friends."

### Chaining

In [148]:
from code.prompts import ClassifierTemplate

In [149]:
ct = ClassifierTemplate(llm)
ct.template_string

'You are a classification expert. Classify the text below into one of the following:\n\n        Class choices: [{classes}]\n\n        Text to classify: {text}\n\n        Classification:\n        '

In [152]:
classes = 'fraud, incorrect form, missing form, need to update information, other'

text = '''Yeah, my name is Michael. I'm calling from the Department of, of Labor. My name um M IC H A E L uh the number here is 60991508926099150892. The reason for the call is that we received the claim and just wanted to review it. Somebody from hr at this point give us a call back at 6099150892. Can also reach us in the office. 609 uh 6099 oh 9083008 6095 9083006099843008609984300.
'''
input_dict = {'classes': classes, 'text': text}
print(ct.process(input_dict))

 other


In [153]:
text = '''Steven C that's S T E P H E N C middle initial player, T H A Y er uh your book publishing at um email address is your book publishing inc all one word, your book publish inc at gmail dot com. I need to change the bank account from to which you send the money of royalties.'''

input_dict = {'classes': classes, 'text': text}
print(ct.process(input_dict))

 incorrect form


In [154]:
text = '''My name is Ahmad Hussein. Uh My email address is Ahmad A H M E D I S MA I L 0777 at hotmail dot com. My phone number is 2709922176. I have received a letter from the IRS saying that I made over 325,000 in 2021 um as in money and I am looking into my seller account. I only opened my account in 2022. So I have no information about making me making all that money in 2021. So I've tried to contact through the Amazon seller support, but I don't have, I don't uh right now all have Amazon seller support in the US, so I cannot contact them. So the IRS told me to call you guys and see what is the amount reported here is for and try to rectify it or see if it was an error or what happened with it. So I would like to call back. Thank you.'''

input_dict = {'classes': classes, 'text': text}
print(ct.process(input_dict))

 incorrect form


In [155]:
text = '''Hi, my name is John Hayes. J O H N H A Y E S and my email is accounting AC C O UN T I N G at D as in David, F as in Franklin, M as in Michael Tole, T 00 L W O R K S. So the full address is accounting at D F M tool works dot com. And I have a question regarding a 10 99 K issue regarding the gross amount paid on the or what was stated on the statement. And I was hoping to speak to a sales or a service representative regarding this issue. If someone please call me back, my number is 6306007023 or you can also reach me at my email. I'll repeat that again. It's accounting at D F M tool works dot com. And I have a question regarding the gross amount stated on that statement for 2022. If someone please contact me at their earliest convenience, that'd be greatly appreciated. Thank you. Bye.'''

input_dict = {'classes': classes, 'text': text}
print(ct.process(input_dict))

 incorrect form
